# 영상 받아서 이미지 한장씩 저장하기

In [177]:
import cv2
import os

filepath = './59.mp4'
video = cv2.cv2.VideoCapture(filepath)

if not video.isOpened():
    print("Could not Open :",filepath)
    exit(0)
    
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 1881
width : 1280
height : 720
fps : 30.0


In [178]:
new_path = './new_image/' # 이미지가 담길 폴더
import cv2
vidcap = cv2.VideoCapture(filepath)  # 영상이 있는 경로 
count = 0 

while(vidcap.isOpened()): 
    ret, image = vidcap.read() # 이미지 사이즈 960x540으로 변경 
    if ret:
        image = cv2.resize(image, (960, 540)) # 30프레임당 하나씩 이미지 추출 
        if(int(vidcap.get(1)) % 30 == 0): 
            print('Saved frame number : ' + str(int(vidcap.get(1)))) # 추출된 이미지가 저장되는 경로 
            cv2.imwrite(new_path+"frame%d.png" % count, image) #
            print('Saved frame%d.jpg' % count) 
            count += 1
    else:
        break
        
vidcap.release()

Saved frame number : 30
Saved frame0.jpg
Saved frame number : 60
Saved frame1.jpg
Saved frame number : 90
Saved frame2.jpg
Saved frame number : 120
Saved frame3.jpg
Saved frame number : 150
Saved frame4.jpg
Saved frame number : 180
Saved frame5.jpg
Saved frame number : 210
Saved frame6.jpg
Saved frame number : 240
Saved frame7.jpg
Saved frame number : 270
Saved frame8.jpg
Saved frame number : 300
Saved frame9.jpg
Saved frame number : 330
Saved frame10.jpg
Saved frame number : 360
Saved frame11.jpg
Saved frame number : 390
Saved frame12.jpg
Saved frame number : 420
Saved frame13.jpg
Saved frame number : 450
Saved frame14.jpg
Saved frame number : 480
Saved frame15.jpg
Saved frame number : 510
Saved frame16.jpg
Saved frame number : 540
Saved frame17.jpg
Saved frame number : 570
Saved frame18.jpg
Saved frame number : 600
Saved frame19.jpg
Saved frame number : 630
Saved frame20.jpg
Saved frame number : 660
Saved frame21.jpg
Saved frame number : 690
Saved frame22.jpg
Saved frame number : 720

# 연속된 이미지 불러와서 검출 -> 분류 -> 시각화

- 연속된 이미지 파일명 : 20200602_1.jpg (이런식으로 파일명을 변경함) 
- 여기서 검출된 신호등 20200602_1_1, 20200602_1_2 ... 이런식으로

In [179]:
import cv2
import matplotlib.pyplot as plt

## green yellow red 모델들

# 파일명 시작 숫자 설정
seq = 1 # 채번 변수 통일 20220308 by yjlim

# 추출 이미지 저장 위치 설정
# 이미지 한장으로 테스트 하는 거라서 디렉토리 분류 할 필요 없이 한 폴더에 다 담기
path = './vi_data2/'

first_detect = []

##### 1. green #####
def green(img, name, X, Y, W, H):
    global seq, path_g, path
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    name = name.split('.')
    
    # ROI 지정
    roi = B[Y:Y+H, X:X+W]
    roi2 = roi.copy()    

    ## 초록색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (43, 40, 120)
    up = (95, 255, 255)

    # 범위내의 픽셀들은 흰색, 나머지 검은색
    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    
    # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion2 = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)    
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)
    

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 8) and (h >= 8) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))
            
    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - 2 * length - 2 * interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,\
                        X + a[0] - 2 * length - 2 * interval_x:X + a[0] + length + interval_x]
                print(img + '///G' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y, a[1] + a[3] + interval_y, X + a[0] - 2 * length - 2 * interval_x, X + a[0] + length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 2. yellow #####
def yellow(img, name, X, Y, W, H):
    global seq, path_y, path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H,X:X+W]
    roi2 = roi.copy()

    ## 노랑색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (10, 10, 110)
    up = (31, 255, 255)
    # low = (20, 20, 100)
    # up = (30, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    
    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 8) and (h >= 8) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - length - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - length - interval_x:X + a[0] + 2 * length + interval_x]
                
                print(img + '///Y' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y,X + a[0] - length - interval_x, X + a[0] + 2 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 3. red #####
def red(img, name, X, Y, W, H):
    global seq, path_r,path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H,X:X+W]
    roi2 = roi.copy()

    ## 빨강색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (0, 30,50)
    up = (10, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    # 색 영역 두번째 추가
    low3 = (150, 40, 50)
    up3 = (180, 255, 255)
    
    roi3_mask = cv2.inRange(roi2_hsv, low3, up3)
    roi3_result = cv2.bitwise_and(roi2, roi2, mask = roi3_mask)
    
    # 색 영역 1 + 색 영역 2  
    result_red = roi2_result + roi3_result

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(result_red, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(result_red, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 8) and (h >= 8) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3)
            interval_y = int(a[3] / 4)
            
            if a[1] - interval_y > 0 and X + a[0] - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - interval_x:X + a[0] + 3 * length + interval_x]
                print(img + '///R' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y, X + a[0] - interval_x, X + a[0] + 3 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] +'_'+ str(seq) + '.jpg', real3)
                seq = seq + 1

In [180]:
import sys
import glob
import cv2

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
# 연속된 이미지가 담긴 폴더
#img_files = glob.glob('./video/*.jpg')
img_files = glob.glob('.\\new_image\\*.png')
img_files.sort() # 순서대로 정렬

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
    
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
    if cv2.waitKey(1000) == 27:     
        break

    index += 1      
    
    # ROI 지정, x,y,w,h ,N 설정 완료
    x1 = 300
    y1 = 0
    w1 = 1400
    h1 = 500
    
    green(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    yellow(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    red(filepath, name, X = x1, Y = y1, W = w1, H = h1)


.\new_image\frame0.png///G1.jpg
.\new_image\frame0.png///Y2.jpg
.\new_image\frame0.png///R3.jpg
.\new_image\frame0.png///R4.jpg
.\new_image\frame0.png///R5.jpg
.\new_image\frame1.png///Y6.jpg
.\new_image\frame1.png///Y7.jpg
.\new_image\frame1.png///Y8.jpg
.\new_image\frame1.png///Y9.jpg
.\new_image\frame10.png///G10.jpg
.\new_image\frame10.png///G11.jpg
.\new_image\frame10.png///Y12.jpg
.\new_image\frame11.png///Y13.jpg
.\new_image\frame11.png///Y14.jpg
.\new_image\frame12.png///Y15.jpg
.\new_image\frame12.png///R16.jpg
.\new_image\frame13.png///R17.jpg
.\new_image\frame13.png///R18.jpg
.\new_image\frame13.png///R19.jpg
.\new_image\frame13.png///R20.jpg
.\new_image\frame13.png///R21.jpg
.\new_image\frame13.png///R22.jpg
.\new_image\frame14.png///G23.jpg
.\new_image\frame14.png///G24.jpg
.\new_image\frame14.png///Y25.jpg
.\new_image\frame14.png///R26.jpg
.\new_image\frame15.png///R27.jpg
.\new_image\frame16.png///R28.jpg
.\new_image\frame16.png///R29.jpg
.\new_image\frame16.png///R30.jp

.\new_image\frame37.png///Y251.jpg
.\new_image\frame37.png///Y252.jpg
.\new_image\frame37.png///Y253.jpg
.\new_image\frame37.png///R254.jpg
.\new_image\frame37.png///R255.jpg
.\new_image\frame37.png///R256.jpg
.\new_image\frame37.png///R257.jpg
.\new_image\frame37.png///R258.jpg
.\new_image\frame37.png///R259.jpg
.\new_image\frame37.png///R260.jpg
.\new_image\frame38.png///Y261.jpg
.\new_image\frame38.png///Y262.jpg
.\new_image\frame38.png///Y263.jpg
.\new_image\frame38.png///R264.jpg
.\new_image\frame38.png///R265.jpg
.\new_image\frame38.png///R266.jpg
.\new_image\frame38.png///R267.jpg
.\new_image\frame39.png///Y268.jpg
.\new_image\frame39.png///Y269.jpg
.\new_image\frame39.png///Y270.jpg
.\new_image\frame39.png///R271.jpg
.\new_image\frame39.png///R272.jpg
.\new_image\frame39.png///R273.jpg
.\new_image\frame39.png///R274.jpg
.\new_image\frame39.png///R275.jpg
.\new_image\frame4.png///Y276.jpg
.\new_image\frame4.png///Y277.jpg
.\new_image\frame4.png///Y278.jpg
.\new_image\frame4.png/

.\new_image\frame56.png///G490.jpg
.\new_image\frame56.png///Y491.jpg
.\new_image\frame56.png///Y492.jpg
.\new_image\frame56.png///Y493.jpg
.\new_image\frame56.png///Y494.jpg
.\new_image\frame56.png///Y495.jpg
.\new_image\frame56.png///Y496.jpg
.\new_image\frame56.png///Y497.jpg
.\new_image\frame56.png///R498.jpg
.\new_image\frame56.png///R499.jpg
.\new_image\frame56.png///R500.jpg
.\new_image\frame56.png///R501.jpg
.\new_image\frame56.png///R502.jpg
.\new_image\frame56.png///R503.jpg
.\new_image\frame57.png///Y504.jpg
.\new_image\frame57.png///Y505.jpg
.\new_image\frame57.png///Y506.jpg
.\new_image\frame57.png///Y507.jpg
.\new_image\frame57.png///Y508.jpg
.\new_image\frame57.png///Y509.jpg
.\new_image\frame57.png///Y510.jpg
.\new_image\frame57.png///R511.jpg
.\new_image\frame57.png///R512.jpg
.\new_image\frame57.png///R513.jpg
.\new_image\frame57.png///R514.jpg
.\new_image\frame57.png///R515.jpg
.\new_image\frame58.png///Y516.jpg
.\new_image\frame58.png///Y517.jpg
.\new_image\frame58.

In [181]:
import os
import cv2
import numpy as np
import random
from PIL import Image

from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf

model = tf.keras.models.load_model('traffic_light_0315_2.h5')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 11, 41, 16)        1216      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 5, 20, 16)        0         
 2D)                                                             
                                                                 
 dropout_15 (Dropout)        (None, 5, 20, 16)         0         
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 18, 32)         4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 1, 9, 32)         0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 1, 9, 32)         

In [182]:
# 검출된 신호등 이미지를 입력 값으로 만들기
Path2 = 'vi_data2/'
file_list = os.listdir(Path2)
file_list.sort()
if ".DS_Store" in file_list:
    file_list.remove(".DS_Store")

data2 = []

for A in file_list:
    if A == ".DS_Store":
        continue
    try:
        img2 = cv2.imread(Path2+A)
        res2 = cv2.resize(img2, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(Path2+A,res2)
    except Exception as e:
        print(str(e))
    
for A in file_list:
    if A == ".DS_Store":
        continue
    try:
        img3 = Image.open(Path2 + A)
        data2.append(np.array(img3))
        img3.close()
    except Exception as e:
        print(str(e))

In [183]:
# data2

In [184]:
data2 = np.asarray(data2) / 255.0
prediction = model.predict(data2)

In [185]:
prediction

array([[0.76240534, 0.0955159 , 0.14207877],
       [0.80938184, 0.07747193, 0.11314628],
       [0.4256429 , 0.21354415, 0.36081296],
       ...,
       [0.03063701, 0.02873145, 0.9406315 ],
       [0.42450503, 0.16821955, 0.4072755 ],
       [0.07361898, 0.00619287, 0.9201881 ]], dtype=float32)

In [186]:
len(first_detect)

537

In [187]:
light_order = {0:"green",1:"red",2:"yellow",3:"unknown"}

# 신호등 파일명과 색을 한 리스트에 담기 
detect_img = [] 

for i, x in enumerate(prediction):
    count = 0
    for y in x:
        if y > 0.95:    # 95퍼센트 이상이면 인식
            count += 1
            
    if count > 0:
        print(str(i)+"번째 신호등: " + file_list[i] + " : " + light_order[np.argmax(x)])
        detect_img.append([file_list[i],light_order[np.argmax(x)]])
    else:
        print(str(i)+"번째 신호등: " + file_list[i] + " : " + light_order[3])

0번째 신호등: frame0_1.jpg : unknown
1번째 신호등: frame0_2.jpg : unknown
2번째 신호등: frame0_3.jpg : unknown
3번째 신호등: frame0_4.jpg : unknown
4번째 신호등: frame0_5.jpg : unknown
5번째 신호등: frame10_10.jpg : unknown
6번째 신호등: frame10_11.jpg : unknown
7번째 신호등: frame10_12.jpg : unknown
8번째 신호등: frame11_13.jpg : unknown
9번째 신호등: frame11_14.jpg : yellow
10번째 신호등: frame12_15.jpg : unknown
11번째 신호등: frame12_16.jpg : unknown
12번째 신호등: frame13_17.jpg : unknown
13번째 신호등: frame13_18.jpg : unknown
14번째 신호등: frame13_19.jpg : unknown
15번째 신호등: frame13_20.jpg : unknown
16번째 신호등: frame13_21.jpg : unknown
17번째 신호등: frame13_22.jpg : unknown
18번째 신호등: frame14_23.jpg : green
19번째 신호등: frame14_24.jpg : green
20번째 신호등: frame14_25.jpg : unknown
21번째 신호등: frame14_26.jpg : unknown
22번째 신호등: frame15_27.jpg : unknown
23번째 신호등: frame16_28.jpg : unknown
24번째 신호등: frame16_29.jpg : unknown
25번째 신호등: frame16_30.jpg : unknown
26번째 신호등: frame16_31.jpg : unknown
27번째 신호등: frame17_32.jpg : unknown
28번째 신호등: frame17_33.jpg : unknown
29번째 신호등: 

In [188]:
detect_img

[['frame11_14.jpg', 'yellow'],
 ['frame14_23.jpg', 'green'],
 ['frame14_24.jpg', 'green'],
 ['frame17_35.jpg', 'red'],
 ['frame1_7.jpg', 'green'],
 ['frame23_70.jpg', 'green'],
 ['frame24_84.jpg', 'green'],
 ['frame24_87.jpg', 'red'],
 ['frame25_101.jpg', 'red'],
 ['frame25_103.jpg', 'red'],
 ['frame26_116.jpg', 'red'],
 ['frame26_117.jpg', 'red'],
 ['frame27_125.jpg', 'red'],
 ['frame28_136.jpg', 'red'],
 ['frame28_137.jpg', 'red'],
 ['frame29_144.jpg', 'red'],
 ['frame29_145.jpg', 'red'],
 ['frame30_165.jpg', 'red'],
 ['frame30_166.jpg', 'red'],
 ['frame31_173.jpg', 'red'],
 ['frame31_174.jpg', 'red'],
 ['frame32_179.jpg', 'yellow'],
 ['frame32_187.jpg', 'red'],
 ['frame32_188.jpg', 'red'],
 ['frame33_202.jpg', 'red'],
 ['frame34_213.jpg', 'red'],
 ['frame34_215.jpg', 'red'],
 ['frame34_216.jpg', 'red'],
 ['frame35_223.jpg', 'yellow'],
 ['frame35_230.jpg', 'red'],
 ['frame35_231.jpg', 'red'],
 ['frame35_232.jpg', 'red'],
 ['frame36_248.jpg', 'red'],
 ['frame36_249.jpg', 'red'],
 ['fr

In [189]:
print(len(file_list),len(first_detect)) # 모델에 적용한 이미지 39개, 이미지 검출 코드에서 나온거 42개 개수가 다름

537 537


In [190]:
first_detect[1][1]

'frame0_2'

In [191]:
xy = []
for i in range(0, len(detect_img)):
    for j in range(0, len(first_detect)):

        if first_detect[j][1] == detect_img[i][0].split(".")[0]:
            xy.append([first_detect[j][1], first_detect[j][0], detect_img[i][1]])
    #print(detect_img[i][0].split(".")[0])

In [192]:
xy

[['frame11_14', [382, 403, 755, 810], 'yellow'],
 ['frame14_23', [154, 173, 313, 364], 'green'],
 ['frame14_24', [148, 166, 481, 529], 'green'],
 ['frame17_35', [328, 348, 883, 933], 'red'],
 ['frame1_7', [443, 455, 286, 325], 'green'],
 ['frame23_70', [380, 401, 925, 980], 'green'],
 ['frame24_84', [473, 486, 897, 944], 'green'],
 ['frame24_87', [312, 325, 470, 503], 'red'],
 ['frame25_101', [321, 335, 702, 752], 'red'],
 ['frame25_103', [297, 311, 460, 496], 'red'],
 ['frame26_116', [317, 332, 745, 803], 'red'],
 ['frame26_117', [280, 298, 450, 494], 'red'],
 ['frame27_125', [260, 278, 436, 480], 'red'],
 ['frame28_136', [280, 300, 883, 949], 'red'],
 ['frame28_137', [230, 245, 426, 473], 'red'],
 ['frame29_144', [309, 324, 467, 506], 'red'],
 ['frame29_145', [191, 212, 409, 467], 'red'],
 ['frame30_165', [303, 315, 466, 499], 'red'],
 ['frame30_166', [152, 176, 394, 460], 'red'],
 ['frame31_173', [299, 312, 465, 498], 'red'],
 ['frame31_174', [115, 142, 380, 452], 'red'],
 ['frame32

In [193]:
array = []
for i in range(0, len(xy)):
    coor = xy[i][1]
    color = xy[i][2]
    for j in range(i+1, len(xy)):
        if xy[i][0] == xy[j][0]:
            coor = xy[j][1]
            color = xy[j][2]
    if [xy[i][0], coor, color] not in array:
        array.append([xy[i][0], coor, color])
    #print(xy[i])

In [194]:
array

[['frame11_14', [382, 403, 755, 810], 'yellow'],
 ['frame14_23', [154, 173, 313, 364], 'green'],
 ['frame14_24', [148, 166, 481, 529], 'green'],
 ['frame17_35', [328, 348, 883, 933], 'red'],
 ['frame1_7', [443, 455, 286, 325], 'green'],
 ['frame23_70', [380, 401, 925, 980], 'green'],
 ['frame24_84', [473, 486, 897, 944], 'green'],
 ['frame24_87', [312, 325, 470, 503], 'red'],
 ['frame25_101', [321, 335, 702, 752], 'red'],
 ['frame25_103', [297, 311, 460, 496], 'red'],
 ['frame26_116', [317, 332, 745, 803], 'red'],
 ['frame26_117', [280, 298, 450, 494], 'red'],
 ['frame27_125', [260, 278, 436, 480], 'red'],
 ['frame28_136', [280, 300, 883, 949], 'red'],
 ['frame28_137', [230, 245, 426, 473], 'red'],
 ['frame29_144', [309, 324, 467, 506], 'red'],
 ['frame29_145', [191, 212, 409, 467], 'red'],
 ['frame30_165', [303, 315, 466, 499], 'red'],
 ['frame30_166', [152, 176, 394, 460], 'red'],
 ['frame31_173', [299, 312, 465, 498], 'red'],
 ['frame31_174', [115, 142, 380, 452], 'red'],
 ['frame32

# array에 담긴 파일명으로 원천 이미지에 시각화

- path 경로에서 원천 이미지 찾아서 그 이미지에다가 rectangle 그리는 거여서 덮어씌워짐  
- 덮어씌워지니까 원천 이미지는 꼭 백업해둘것

In [195]:
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 

for a,b,c in array:
    print(a,b,c)
    imgname = a.split('_')[0] + '.png'
    print(path,imgname)

frame11_14 [382, 403, 755, 810] yellow
./new_image/ frame11.png
frame14_23 [154, 173, 313, 364] green
./new_image/ frame14.png
frame14_24 [148, 166, 481, 529] green
./new_image/ frame14.png
frame17_35 [328, 348, 883, 933] red
./new_image/ frame17.png
frame1_7 [443, 455, 286, 325] green
./new_image/ frame1.png
frame23_70 [380, 401, 925, 980] green
./new_image/ frame23.png
frame24_84 [473, 486, 897, 944] green
./new_image/ frame24.png
frame24_87 [312, 325, 470, 503] red
./new_image/ frame24.png
frame25_101 [321, 335, 702, 752] red
./new_image/ frame25.png
frame25_103 [297, 311, 460, 496] red
./new_image/ frame25.png
frame26_116 [317, 332, 745, 803] red
./new_image/ frame26.png
frame26_117 [280, 298, 450, 494] red
./new_image/ frame26.png
frame27_125 [260, 278, 436, 480] red
./new_image/ frame27.png
frame28_136 [280, 300, 883, 949] red
./new_image/ frame28.png
frame28_137 [230, 245, 426, 473] red
./new_image/ frame28.png
frame29_144 [309, 324, 467, 506] red
./new_image/ frame29.png
frame2

In [196]:
# 원천이미지 있는 디렉토리 경로
# 해당 디렉토리 백업해두기
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 

for a,b,c in array:
    print(a,b,c)
    imgname = a.split('_')[0]+'.png'
    img = cv2.imread(path+imgname)
    y,h,x,w = b
    w = w - x
    h = h - y
    text = c 
    org = (x, y - 8)
    
    if x > 0 and y > 0 and w > 0 and h > 0:
        cv2.rectangle(img, (x, y, w, h), (139, 255, 224), 2)
        cv2.putText(img, text, org, font, 0.7, (139, 255, 224), 2)
        
    # 기존 원천 이미지에 덮어쓰기 되는거니까 주의!!!
    cv2.imwrite(path+imgname, img)


    #print(a,b,c)

frame11_14 [382, 403, 755, 810] yellow
frame14_23 [154, 173, 313, 364] green
frame14_24 [148, 166, 481, 529] green
frame17_35 [328, 348, 883, 933] red
frame1_7 [443, 455, 286, 325] green
frame23_70 [380, 401, 925, 980] green
frame24_84 [473, 486, 897, 944] green
frame24_87 [312, 325, 470, 503] red
frame25_101 [321, 335, 702, 752] red
frame25_103 [297, 311, 460, 496] red
frame26_116 [317, 332, 745, 803] red
frame26_117 [280, 298, 450, 494] red
frame27_125 [260, 278, 436, 480] red
frame28_136 [280, 300, 883, 949] red
frame28_137 [230, 245, 426, 473] red
frame29_144 [309, 324, 467, 506] red
frame29_145 [191, 212, 409, 467] red
frame30_165 [303, 315, 466, 499] red
frame30_166 [152, 176, 394, 460] red
frame31_173 [299, 312, 465, 498] red
frame31_174 [115, 142, 380, 452] red
frame32_179 [320, 333, 667, 703] yellow
frame32_187 [360, 375, 541, 599] red
frame32_188 [90, 120, 374, 446] red
frame33_202 [82, 109, 371, 440] red
frame34_213 [356, 369, 560, 593] red
frame34_215 [297, 309, 468, 496] r

# 원천 이미지 연속 출력 (검출된 내용 반영)

In [197]:
import sys
import glob
import cv2

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
img_files = glob.glob('.\\new_image\\*.png')
img_files.sort()

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index<count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
	# ESC가 입력되면 break
    cv2.imshow('video_test', img)
    if cv2.waitKey(1000) == 27:     
        break

	# index가 이미지 리스트보다 커지거나 같아지면 다시 0으로
    index += 1      
    #if index >= count :
        #index = 0 

cv2.waitKey(0)    
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# 동영상으로 저장

In [198]:
import numpy as np
import glob
 
img_array = []
for filename in glob.glob('.\\new_image\\*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('video_test.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()